In [1]:
#Imports:

import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import numpy as np
import re


In [2]:
c = requests.get('https://imoveis.mercadolivre.com.br/aluguel/').content

In [3]:
zonas = ['norte','sul','leste','oeste']
URL='https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-zona-{}/{}'
paginas=['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', 
           '_Desde_241', '_Desde_289']


In [4]:
# Functions
def scrap(url):
    c=requests.get(url).content
    soup=BeautifulSoup(c)
    aluguel_tags = soup.find_all('span', {'class':'price-tag-fraction'})
    alugueis_lista = [p.text for p in aluguel_tags]
    enderecos_tags = soup.find_all('span', {'class':'ui-search-item__group__element'})
    enderecos_lista=[end.text for end in enderecos_tags]
    enderecos_lista=enderecos_lista[1::2]
    area_quartos_tags = soup.find_all('ul', {'class':'ui-search-card-attributes'})
    area_quartos_lista=[aq.text for aq in area_quartos_tags]
    df=convert_df(zona, area_quartos_lista, enderecos_lista, alugueis_lista)
    return df

def convert_df(zona, area_quartos_lista, enderecos_lista, alugueis_lista):
    zonas_lista = [zona]*len(alugueis_lista)
    data = {'zona': zonas_lista,
            'area_quartos': area_quartos_lista,
            'enderecos_lista':enderecos_lista,
            'aluguel': alugueis_lista}
    df = pd.DataFrame(data)
    return df

In [6]:
lista_dfs = []
for zona in zonas:
    for pg in paginas:
        url='https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-{}/{}'.format(zona, pg)
        print(url)
        df=scrap(url)
        lista_dfs.append(df)
        sleep(1)

https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_49
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_97
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_145
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_193
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_241
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_289
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_97
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_145
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_193
ht

In [7]:
df=pd.concat(lista_dfs)
df.head()

,zona,area_quartos,enderecos_lista,aluguel
0,norte,18 m² construídos1 quarto,"Rua Amaná, Vila Sabrina, São Paulo Zona Norte,...",770
1,norte,42 m² construídos1 quarto,"Av Do Guacá, Lauzane Paulista, São Paulo Zona ...",1.600
2,norte,80 m² construídos2 quartos,"Av Braz Leme, Santana, São Paulo Zona Norte, S...",2.000
3,norte,68 m² construídos3 quartos,"Avenida José Maria Fernandes, Parque Novo Mund...",1.365
4,norte,28 m² construídos1 quarto,"Rua Alberto Costa, Parque Mandaqui, São Paulo ...",770


In [8]:
df.to_csv('dados_aluguel_ml_sao_paulo.csv', index=False)

In [9]:
# Funções
def extrair_area(entrada):
  """Criar função que possua como entrada um texto e retorne a área contida no
  mesmo. Retornar np.nan se for ausentes. Alguns exemplos:
      >>> extrair_area('79 m² construídos1 quarto')
      >>> 79
      >>> extrair_area('280 m² construídos')
      >>> 280
      >>> extrair_area('2,600 m² construídos'
      >>> 2600
      >>> extrair_area('2 quartos')
      >>> np.nan
      """
  if ' m²' in entrada:
    area = entrada.split(' m²')[0]
    area = area.replace(',', '')
  else:
    area = np.nan
  return area

def extrair_quartos(entrada):
  """Criar função que possua como entrada um texto e retorne uma tupla com a 
  área e o quarto. Retornar np.nan se um dos dois for ausentes. Alguns exemplos:
      >>> extrair_quarto('79 m² construídos1 quarto')
      >>> 1
      >>> extrair_quarto('280 m² construídos')
      >>> np.nan
      >>> extrair_quarto('2,600 m² construídos'
      >>> np.nan
      >>> extrair_quarto('2 quartos')
      >>> 2"""
  if 'quarto' in entrada:
    n_quartos = re.findall('(\d*) quarto', entrada)[0]
  else:
    n_quartos = np.nan
  return n_quartos

In [10]:
df['area']=df['area_quartos'].apply(extrair_area).astype(float)
df['quartos'] = df['area_quartos'].apply(extrair_quartos).astype(float)

df = df.drop('area_quartos', axis=1)
df.head()

,zona,enderecos_lista,aluguel,area,quartos
0,norte,"Rua Amaná, Vila Sabrina, São Paulo Zona Norte,...",770,18.0,1.0
1,norte,"Av Do Guacá, Lauzane Paulista, São Paulo Zona ...",1.600,42.0,1.0
2,norte,"Av Braz Leme, Santana, São Paulo Zona Norte, S...",2.000,80.0,2.0
3,norte,"Avenida José Maria Fernandes, Parque Novo Mund...",1.365,68.0,3.0
4,norte,"Rua Alberto Costa, Parque Mandaqui, São Paulo ...",770,28.0,1.0


In [11]:
df.to_csv('dados_aluguel_ml_sao_paulo_preprocessado.csv', index=False)

In [12]:
for i in range(0,20):
    print(df.iloc[i]['enderecos_lista'])

Rua Amaná, Vila Sabrina, São Paulo Zona Norte, São Paulo
Av Do Guacá, Lauzane Paulista, São Paulo Zona Norte, São Paulo
Av Braz Leme, Santana, São Paulo Zona Norte, São Paulo
Avenida José Maria Fernandes, Parque Novo Mundo, São Paulo Zona Norte, São Paulo
Rua Alberto Costa, Parque Mandaqui, São Paulo Zona Norte, São Paulo
Rua Cabo José Da Silva, Parque Novo Mundo, São Paulo Zona Norte, São Paulo
Rua Aída Gomes Tolêdo, Imirim, São Paulo Zona Norte, São Paulo
Rua Doutor Heitor Nascimento, Nossa Senhora Do Ó, São Paulo Zona Norte, São Paulo
Rua Dona Estela Borges Morato, Vila Siqueira (zona Norte), São Paulo Zona Norte, São Paulo
Rua Pedro Cacunda, Jardim São Paulo, São Paulo Zona Norte, São Paulo
Rua José Debieux, Santana, São Paulo Zona Norte, São Paulo
Rua Ciridião Durval, Vila Paulista, São Paulo Zona Norte, São Paulo
Avenida João Pessoa, Lauzane Paulista, São Paulo Zona Norte, São Paulo
Rua Iwajiro Takahashi, Vila Mazzei, São Paulo Zona Norte, São Paulo
Rua Relíquia, Jardim Das Laran